# Random Numbers

reference: https://towardsdatascience.com/how-to-generate-random-variables-from-scratch-no-library-used-4b71eb3c8dc7

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
def get_seed():
    """Generates random seed using decimal part of system clock"""
    t = time.perf_counter()
    return int((t % 1) * 1e16)

def rand(a=1664525, c=1013904223, m=2**32, size=1, seed=None):
    """Generates Uniform(0,1) samples using linear congruence generator"""
    lcg = lambda x: (a * x + c) % m
    x = get_seed() if seed is None else seed
    u = []
    for i in range(size):
        x = lcg(x)
        u.append(x / m)
    return np.array(u)

def uniform(low=0, high=1, size=1, seed=None):
    """Generates U(low, high) samples"""
    return ((high - low) * rand(seed=seed, size=size) + low)

def randint(low=0, high=2, size=1, seed=None):
    """Generates DU(low, high) samples, i.e. random ints [low, high)"""
    u = uniform(low=low, high=high, size=size, seed=seed)
    return np.floor(u).astype(int)

def choice(array, size=1, seed=None):
    """Randomly samples from given array, with replacement"""
    array = np.array(array)
    r = randint(low=0, high=len(array), size=size, seed=seed)
    return array[r]

def bernoulli(p=0.5, size=1, seed=None):
    """Generates Bernoulli(p) samples"""
    u = rand(size=size, seed=seed)
    return (u <= p).astype(int)

def binomial(n, p=0.5, size=1, seed=None):
    """Generates Binomial(n, p) samples"""
    x = []
    for i in range(size):
        b = bernoulli(p=p, size=n, seed=seed)
        x.append(b.sum())
    return np.array(x)

def poisson(lambda_=1, size=1, seed=None):
    """Generates Poisson(lambda) samples using Knuth algorithm"""
    array = []
    L = np.exp(-lambda_)
    for j in range(size):
        x = 0
        p = 1
        while p > np.exp(-lambda_):
            x += 1
            p = p * rand(seed=seed)
        array.append(x - 1)
    return np.array(array)

def exponential(lambda_=1, size=1, seed=None):
    """Generates Exp(lambda) samples using Inverse Transform Sampling"""
    u = rand(seed=seed, size=size)
    x = - (1 / lambda_) * np.log(1 - u)
    return x

def normal(mu=0, sigma=1, size=1, seed=None):
    """Generates N(mu, sigma) samples using Box-Muller Transform"""
    u1 = rand(seed=seed, size=size)
    u2 = rand(seed=seed, size=size)
    z0 = np.sqrt(-2 * np.log(u1)) * np.cos(2 * np.pi * u2)
    z1 = np.sqrt(-2 * np.log(u1)) * np.sin(2 * np.pi * u2)
    return z0 * sigma + mu

def _gen_and_plot(dist, **kwargs):
    x = dist(**kwargs)
    print(f'--------- {dist.__name__} ---------')
    print(f'params = {kwargs}')
    if dist.__name__ != 'choice':
        print(f'mean = {x.mean()}   var = {x.var()}')
    plt.hist(x)
    plt.title(f'{dist.__name__}{kwargs}')
    plt.show()
    print()
    print()

In [ ]:
size = 1000
_gen_and_plot(rand, size=size)
_gen_and_plot(uniform, size=size, low=1, high=10)
_gen_and_plot(randint, size=size, low=1, high=5)
_gen_and_plot(choice, size=size, array=['a', 'b', 'b', 'a'])
_gen_and_plot(bernoulli, size=size, p=0.2)
_gen_and_plot(binomial, size=size, n=10, p=0.5)
_gen_and_plot(poisson, size=size, lambda_=3)
_gen_and_plot(exponential, size=size, lambda_=1)
_gen_and_plot(normal, size=size, mu=0, sigma=1)